<div style="display: flex; align-items: center;">
    <h1>Optimizing parameters in a WOFOST crop model using <code>diffWOFOST</code></h1>
    <img src="https://raw.githubusercontent.com/WUR-AI/diffWOFOST/refs/heads/main/docs/logo/diffwofost.png" width="150" style="margin-left: 20px;">
</div>

In this Jupyter notebook we will demonstrate how to optimize parameters in a
differentiable model. There are two sections:
1. Leaf dynamics
2. Root dynamics

## 1. Leaf dynamics

In this section, we will demonstrate how to optimize two parameters `TWDI` and `SPAN` in
leaf_dynamics model using a differentiable version of leaf_dynamics.
The optimization will be done using the Adam optimizer from `torch.optim`.

### 1.1 software requirements

To run this notebook, we need to install the `diffwofost`; the diffrentiable
version of WOFOST models. Since the package is constantly under development, make
sure you have the latest version of `diffwofost` installed in your
python environment. You can install it using pip:

```bash
pip install diffwofost
```

In [1]:
# ---- import libraries ----
import copy
import torch
import numpy
import yaml
from pathlib import Path
from diffwofost.physical_models.utils import EngineTestHelper
from diffwofost.physical_models.utils import prepare_engine_input

In [2]:
# ---- disable a warning: this will be fixed in the future ----
import warnings
warnings.filterwarnings("ignore", message="To copy construct from a tensor.*")

### 1.2. Data

A test dataset of `LAI` (Leaf area index, including stem and pod area) and
`TWLV` (Dry weight of total leaves (living + dead)) will be used to optimize
parametesr `TWDI` (total initial dry weight) and `SPAN` (life span of leaves).

The data is stored in PCSE tests folder, and can be doewnloded from PCSE repsository.
You can select any of the files related to `leaf_dynamics` model with a file name that follwos the pattern
`test_leafdynamics_wofost72_*.yaml`. Each file contains different data depending on the locatin and crop type.
For example, you can download the file "test_leafdynamics_wofost72_01.yaml" as:

```bash
wget https://raw.githubusercontent.com/ajwdewit/pcse/refs/heads/master/tests/test_data/test_leafdynamics_wofost72_01.yaml
```

We also need to download a config file to be able to run each crop module. This will change in the future versions. To donwload the config file, you can use the following command:

```bash
wget https://raw.githubusercontent.com/WUR-AI/diffWOFOST/refs/heads/main/tests/physical_models/test_data/WOFOST_Leaf_Dynamics.conf
```

In [3]:
# ---- Check the path to the files that are downloaded as explained above ----
test_data_path = "test_leafdynamics_wofost72_01.yaml"
config_path = "WOFOST_Leaf_Dynamics.conf"

In [4]:
# ---- Here we read the test data ans et some variables ----
(crop_model_params_provider, weather_data_provider, agro_management_inputs, external_states) = (
    prepare_engine_input(test_data_path, ["SPAN", "TDWI", "TBASE", "PERDL", "RGRLAI"])
)

expected_results = yaml.safe_load(open(test_data_path))["ModelResults"]
expected_lai_twlv = torch.tensor(
    [[float(item["LAI"]), float(item["TWLV"])] for item in expected_results], dtype=torch.float32
).unsqueeze(0) # shape: [1, time_steps, 2]

# ---- dont change this: in this config file we specified the diffrentiable version of leaf_dynamics ----
config_path = str(Path(config_path).resolve()) 

### 1.3. Helper classes/functions

The model parameters shoudl stay in a valid range. To ensure this, we will use
`BoundedParameter` class with (min, max) and initial values for each
parameter. You might change these values depending on the crop type and
location. But dont use a very small range, otherwise gradiants will be very
small and the optimization will be very slow.

In [5]:
# ---- Adjust the values if needed  ----
TDWI_MIN, TDWI_MAX, TDWI_INIT = (0.0, 1.0, 0.40)
SPAN_MIN, SPAN_MAX, SPAN_INIT = (10.0, 60.0, 20.0)

# ---- Helper for bounded parameters ----
class BoundedParameter(torch.nn.Module):
    def __init__(self, low, high, init_value):
        super().__init__()
        self.low = low
        self.high = high

        # Normalize to [0, 1]
        init_norm = (init_value - low) / (high - low)

        # Clamp to avoid logit infinities
        init_norm = torch.clamp(
            torch.tensor(init_norm, dtype=torch.float32),
            1e-6, 1 - 1e-6
        )

        # Parameter in raw logit space
        self.raw = torch.nn.Parameter(torch.logit(init_norm))

    def forward(self):
        return self.low + (self.high - self.low) * torch.sigmoid(self.raw)


Another helper class is OptDiffLeafDynamics which is a subclass of torch.nn.Module. 
We use this class to wrap the leaf_dynamics model and make it easier to use with torch optimizers.

In [6]:
# ---- Wrap the model with torch.nn.Module----
class OptDiffLeafDynamics(torch.nn.Module):
    def __init__(self, crop_model_params_provider, weather_data_provider, agro_management_inputs, config_path, external_states):
        super().__init__()
        self.crop_model_params_provider = crop_model_params_provider
        self.weather_data_provider = weather_data_provider
        self.agro_management_inputs = agro_management_inputs
        self.config_path = config_path
        self.external_states = external_states

        # bounded parameters
        self.tdwi = BoundedParameter(TDWI_MIN, TDWI_MAX, init_value=TDWI_INIT)
        self.span = BoundedParameter(SPAN_MIN, SPAN_MAX, init_value=SPAN_INIT)

    def forward(self):
        # currently, copying is needed due to an internal issue in engine
        crop_model_params_provider_ = copy.deepcopy(self.crop_model_params_provider)
        external_states_ = copy.deepcopy(self.external_states)
        
        tdwi_val = self.tdwi()
        span_val = self.span()
        
        # pass new value of parameters to the model
        crop_model_params_provider_.set_override("TDWI", tdwi_val, check=False)
        crop_model_params_provider_.set_override("SPAN", span_val, check=False)

        engine = EngineTestHelper(
            crop_model_params_provider_,
            self.weather_data_provider,
            self.agro_management_inputs,
            self.config_path,
            external_states_,
        )
        engine.run_till_terminate()
        results = engine.get_output()
        
        return torch.stack(
            [torch.stack([item["LAI"], item["TWLV"]]) for item in results]
        ).unsqueeze(0) # shape: [1, time_steps, 2]

In [7]:
# ----  Create model ---- 
opt_model = OptDiffLeafDynamics(
    crop_model_params_provider,
    weather_data_provider,
    agro_management_inputs,
    config_path,
    external_states,
)

# ----  Optimizer ---- 
optimizer = torch.optim.Adam(opt_model.parameters(), lr=0.1)

# ----  We use relative MAE as loss because there are two outputs with different untis ----  
denom = torch.mean(torch.abs(expected_lai_twlv), dim=1) 

# Training loop (example)
for step in range(101):
    optimizer.zero_grad()
    results = opt_model() 
    mae = torch.mean(torch.abs(results - expected_lai_twlv), dim=1)
    rmae = mae / denom
    loss = rmae.sum()  # example: relative mean absolute error
    loss.backward()
    optimizer.step()

    if step % 10 == 0:
        print(f"Step {step}, Loss {loss.item():.4f}, TDWI {opt_model.tdwi().item():.4f}, SPAN {opt_model.span().item():.4f}")

Step 0, Loss 0.3969, TDWI 0.4242, SPAN 20.8240
Step 10, Loss 0.1626, TDWI 0.4908, SPAN 29.9683
Step 20, Loss 0.0677, TDWI 0.5133, SPAN 37.6622
Step 30, Loss 0.0263, TDWI 0.5156, SPAN 33.6077
Step 40, Loss 0.0658, TDWI 0.5150, SPAN 33.0349
Step 50, Loss 0.0350, TDWI 0.5130, SPAN 35.7802
Step 60, Loss 0.0226, TDWI 0.5010, SPAN 34.4072
Step 70, Loss 0.0444, TDWI 0.5138, SPAN 36.6308
Step 80, Loss 0.0692, TDWI 0.5059, SPAN 31.9481
Step 90, Loss 0.1520, TDWI 0.4953, SPAN 29.3475
Step 100, Loss 0.1420, TDWI 0.4981, SPAN 29.9158


In [8]:
# ---- validate the results using test data ---- 
print(f"Actual TDWI {crop_model_params_provider["TDWI"].item():.4f}, SPAN {crop_model_params_provider["SPAN"].item():.4f}")

Actual TDWI 0.5100, SPAN 35.0000


## 2. Root dynamics 

In this section, we will demonstrate how to optimize two parameters `TWDI` in
root_dynamics model using a differentiable version of root_dynamics.
The optimization will be done using the Adam optimizer from `torch.optim`.

### 2.1 software requirements

To run this notebook, we need to install the `diffwofost`; the diffrentiable
version of WOFOST models. Since the package is constantly under development, make
sure you have the latest version of `diffwofost` installed in your
python environment. You can install it using pip:

```bash
pip install diffwofost
```

In [9]:
# ---- import libraries ----
import copy
import torch
import numpy
import yaml
from pathlib import Path
from diffwofost.physical_models.utils import EngineTestHelper
from diffwofost.physical_models.utils import prepare_engine_input

In [10]:
# ---- disable a warning: this will be fixed in the future ----
import warnings
warnings.filterwarnings("ignore", message="To copy construct from a tensor.*")

### 2.2. Data

A test dataset of `RD` (Current rooting depth) and
`TWRT` (Total weight of roots) will be used to optimize
parametesr `TWDI` (total initial dry weight).

The data is stored in PCSE tests folder, and can be doewnloded from PCSE repsository.
You can select any of the files related to `root_dynamics` model with a file name that follwos the pattern
`test_rootdynamics_wofost72_*.yaml`. Each file contains different data depending on the locatin and crop type.
For example, you can download the file "test_rootdynamics_wofost72_01.yaml" as:

```bash
wget https://raw.githubusercontent.com/ajwdewit/pcse/refs/heads/master/tests/test_data/test_rootdynamics_wofost72_01.yaml
```

We also need to download a config file to be able to run each crop module. This will change in the future versions. To donwload the config file, you can use the following command:

```bash
wget https://raw.githubusercontent.com/WUR-AI/diffWOFOST/refs/heads/main/tests/physical_models/test_data/WOFOST_Root_Dynamics.conf
```

In [11]:
# ---- Check the path to the files that are downloaded as explained above ----
test_data_path = "test_rootdynamics_wofost72_01.yaml"
config_path = "WOFOST_Root_Dynamics.conf"

In [12]:
# ---- Here we read the test data ans et some variables ----
(crop_model_params_provider, weather_data_provider, agro_management_inputs, external_states) = (
    prepare_engine_input(test_data_path, ["RDI", "RRI", "RDMCR", "RDMSOL", "TDWI", "IAIRDU"])
)

expected_results = yaml.safe_load(open(test_data_path))["ModelResults"]
expected_rd_twrt = torch.tensor(
    [[float(item["RD"]), float(item["TWRT"])] for item in expected_results], dtype=torch.float32
).unsqueeze(0) # shape: [1, time_steps, 2]

# ---- dont change this: in this config file we specified the diffrentiable version of root_dynamics ----
config_path = str(Path(config_path).resolve()) 

### 2.3. Helper classes/functions

The model parameters shoudl stay in a valid range. To ensure this, we will use
`BoundedParameter` class with (min, max) and initial values for each
parameter. You might change these values depending on the crop type and
location. But dont use a very small range, otherwise gradiants will be very
small and the optimization will be very slow.

In [13]:
# ---- Adjust the values if needed  ----
TDWI_MIN, TDWI_MAX, TDWI_INIT = (0.0, 1.0, 0.30)

# ---- Helper for bounded parameters ----
class BoundedParameter(torch.nn.Module):
    def __init__(self, low, high, init_value):
        super().__init__()
        self.low = low
        self.high = high

        # Normalize to [0, 1]
        init_norm = (init_value - low) / (high - low)

        # Clamp to avoid logit infinities
        init_norm = torch.clamp(
            torch.tensor(init_norm, dtype=torch.float32),
            1e-6, 1 - 1e-6
        )

        # Parameter in raw logit space
        self.raw = torch.nn.Parameter(torch.logit(init_norm))

    def forward(self):
        return self.low + (self.high - self.low) * torch.sigmoid(self.raw)


In [14]:
# ---- Wrap the model with torch.nn.Module----
class OptDiffLeafDynamics(torch.nn.Module):
    def __init__(self, crop_model_params_provider, weather_data_provider, agro_management_inputs, config_path, external_states):
        super().__init__()
        self.crop_model_params_provider = crop_model_params_provider
        self.weather_data_provider = weather_data_provider
        self.agro_management_inputs = agro_management_inputs
        self.config_path = config_path
        self.external_states = external_states

        # bounded parameters
        self.tdwi = BoundedParameter(TDWI_MIN, TDWI_MAX, init_value=TDWI_INIT)
        
    def forward(self):
        # currently, copying is needed due to an internal issue in engine
        crop_model_params_provider_ = copy.deepcopy(self.crop_model_params_provider)
        external_states_ = copy.deepcopy(self.external_states)
        
        tdwi_val = self.tdwi()
        
        # pass new value of parameters to the model
        crop_model_params_provider_.set_override("TDWI", tdwi_val, check=False)

        engine = EngineTestHelper(
            crop_model_params_provider_,
            self.weather_data_provider,
            self.agro_management_inputs,
            self.config_path,
            external_states_,
        )
        engine.run_till_terminate()
        results = engine.get_output()
        
        return torch.stack(
            [torch.stack([item["RD"], item["TWRT"]]) for item in results]
        ).unsqueeze(0) # shape: [1, time_steps, 2]

In [15]:
# ----  Create model ---- 
opt_model = OptDiffLeafDynamics(
    crop_model_params_provider,
    weather_data_provider,
    agro_management_inputs,
    config_path,
    external_states,
)

# ----  Optimizer ---- 
optimizer = torch.optim.Adam(opt_model.parameters(), lr=0.1)

# ----  We use relative MAE as loss because there are two outputs with different untis ----  
denom = torch.mean(torch.abs(expected_rd_twrt), dim=1) 

# Training loop (example)
for step in range(101):
    optimizer.zero_grad()
    results = opt_model() 
    mae = torch.mean(torch.abs(results - expected_rd_twrt), dim=1)
    rmae = mae / denom
    loss = rmae.sum()  # example: relative mean absolute error
    loss.backward()
    optimizer.step()

    if step % 10 == 0:
        print(f"Step {step}, Loss {loss.item():.4f}, TDWI {opt_model.tdwi().item():.4f}")

Step 0, Loss 0.0000, TDWI 0.3214
Step 10, Loss 0.0000, TDWI 0.5424
Step 20, Loss 0.0000, TDWI 0.4864
Step 30, Loss 0.0000, TDWI 0.5227
Step 40, Loss 0.0000, TDWI 0.5150
Step 50, Loss 0.0000, TDWI 0.5190
Step 60, Loss 0.0000, TDWI 0.5109
Step 70, Loss 0.0000, TDWI 0.5104
Step 80, Loss 0.0000, TDWI 0.5069
Step 90, Loss 0.0000, TDWI 0.5055
Step 100, Loss 0.0000, TDWI 0.5090


In [16]:
# ---- validate the results using test data ---- 
print(f"Actual TDWI {crop_model_params_provider["TDWI"].item():.4f}")

Actual TDWI 0.5100
